In [8]:
import scrapy
import requests
import asyncio
from scrapy.crawler import Crawler
import json
from urllib.parse import quote

In [ ]:
class Configurations:
    def __init__(self, proxy, urls) -> None:
        self.proxy: str = proxy,
        self.urls: list = urls
        pass
    
       
    def create_client(self) -> requests.Session:
        if self.proxy is None:
            raise Exception("no proxy available")
        proxies = {"http": self.proxy}
        headers = {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:137.0) Gecko/20100101 Firefox/137.0"
        }
        client = requests.Session()
        client.proxies.update(proxies)
        return client
    
    
    def check_status(self) -> None:
        client = self.create_client()
        for url in self.urls:
            response = client.get(url)
            return print(response.status_code)
   
        
class NYTimesSpider(scrapy.Spider, Configurations):
    name = "nytimesspider"
    allowed_domains = ["nytimes.com"]
    start_urls = ["https://nytimes.com/section/business/media"]
    
    
    def get_tokens(self):
        session = requests.Session()
        session.proxies.update(proxies)
        session.headers.update(headers)

        r = session.get(url="https://nytimes.com",headers=headers)
        

        token_search = re.search(r'["\']nyt-token["\']\s*:\s*["\']([^"\']+)["\']', r.text)
        app_type_search = re.search(r'["\']nyt-app-type["\']\s*:\s*["\']([^"\']+)["\']', r.text)
        app_type_version = re.search(r'["\']nyt-app-version["\']\s*:\s*["\']([^"\']+)["\']', r.text)
        
        return token_search, app_type_search, app_type_version
        

    def start_requests(self, cursor= None):
        operation_name = "CollectionsQuery"
        variables = {
    "id":"/section/business/media",
    "first":10,
    "exclusionMode":"HIGHLIGHTS_AND_EMBEDDED",
    "isFetchMore":False,
    "isTranslatable":False,
    "isEspanol":False,
    "highlightsListUri":"nyt://per/personalized-list/__null__",
    "highlightsListFirst":0,
    "hasHighlightsList":False,
    "cursor": cursor # None when Opening the page
        }

        extension= {"persistedQuery":
                          {"version":1,"sha256Hash":"8334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8"}
                          }
        var_query = quote(json.dumps(variables))
        extension_query = quote(json.dumps(variables))


        url = f"https://samizdat-graphql.nytimes.com/graphql/v2?operation_name={operation_name}&variables={var_query}&extension={extension_query}"


        yield scrapy.Request(
            url=url,
            headers={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:137.0) Gecko/20100101 Firefox/137.0"},
            callback=self.parse 
            )



    def parse(self, response):
        data = response.json()
        
        collection = data['data']['legacyCollection']['collectionsPage']
        articles = collection['stream']['edges']

        for article in articles:
            yield {
                'headline':article['node']['headline']['default'],  #Need to get text which is in default="headline"
                'summary': article['node']['summary'],
                'url': article['node']['url'],
                'News Source': article['node']['_typename']
            }
        
        #Now we need to parse through the pages
        #The end paramater will be the new start parameter
        start_cursor = collection['stream']['pageInfo']['endCursor']
        if start_cursor:
            yield self.make_next_request(start_cursor)
        
    def make_next_request(self, new_cursor):
        start_request(new_cursor)
        
        
        
    
    

    
    
    
#def main():
    #example = Extractor(["https://www.nytimes.com/media"])
    #process = CrawlerProcess(example.create_client.headers)
    #process.crawl(NYTimesSpider)
    #process.start()
    


#if __name__ == "__main__":
    #main()
        

Building an API query:

https://samizdat-graphql.nytimes.com/graphql/v2?
operationName=CollectionsQuery&
variables={
    "id":"/section/business/media",
    "first":10,
    "exclusionMode":"HIGHLIGHTS_AND_EMBEDDED",
    "isFetchMore":false,
    "isTranslatable":false,
    "isEspanol":false,
    "highlightsListUri":"nyt://per/personalized-list/__null__",
    "highlightsListFirst":0,
    "hasHighlightsList":false,
    "cursor":"YXJyYXljb25uZWN0aW9uOjIwOQ=="}
    &extensions=
    {"persistedQuery":{"version":1,"sha256Hash":"8334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8"}}

https://samizdat-graphql.nytimes.com/graphql/v2?operationName=CollectionsQuery&variables=%7B%22id%22%3A%22%2Fsection%2Fbusiness%2Fmedia%22%2C%22first%22%3A10%2C%22exclusionMode%22%3A%22HIGHLIGHTS_AND_EMBEDDED%22%2C%22isFetchMore%22%3Afalse%2C%22isTranslatable%22%3Afalse%2C%22isEspanol%22%3Afalse%2C%22highlightsListUri%22%3A%22nyt%3A%2F%2Fper%2Fpersonalized-list%2F__null__%22%2C%22highlightsListFirst%22%3A0%2C%22hasHighlightsList%22%3Afalse%2C%22cursor%22%3A%22YXJyYXljb25uZWN0aW9uOjIwOQ%3D%3D%22%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%228334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8%22%7D%7D

In [ ]:
import requests
from urllib.parse import quote
import json

def scrape_nytimes_with_session():
    session = requests.Session()
    
    # Step 1: Get the main page to establish cookies and get the nyt-token
    print("Getting main page to establish session...")
    main_response = session.get("https://www.nytimes.com/section/business/media")
    
    # Step 2: Extract the nyt-token from the page (it's usually in a script tag)
    # You'll need to parse the HTML to find the token, or use the one from your curl
    
    # Step 3: Set up headers exactly like the browser
    headers = {
        "accept": "*/*",
        "accept-language": "en-US,en;q=0.9",
        "content-type": "application/json",
        "nyt-app-type": "project-vi",
        "nyt-app-version": "0.0.5",
        "nyt-token": nyt_token,
        "origin": "https://www.nytimes.com",
        "priority": "u=1, i",
        "referer": "https://www.nytimes.com/",
        "sec-ch-ua": '"Google Chrome";v="134", "Chromium";v="134", "Not_A Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"macOS"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.6998.205 Safari/537.36",
        "x-nyt-internal-meter-override": "undefined"
    }
    
    # Step 4: Make the GraphQL request
    url = "https://samizdat-graphql.nytimes.com/graphql/v2?operationName=CollectionsQuery&variables=%7B%22id%22%3A%22%2Fsection%2Fbusiness%2Fmedia%22%2C%22first%22%3A10%2C%22exclusionMode%22%3A%22HIGHLIGHTS_AND_EMBEDDED%22%2C%22isFetchMore%22%3Afalse%2C%22isTranslatable%22%3Afalse%2C%22isEspanol%22%3Afalse%2C%22highlightsListUri%22%3A%22nyt%3A%2F%2Fper%2Fpersonalized-list%2F__null__%22%2C%22highlightsListFirst%22%3A0%2C%22hasHighlightsList%22%3Afalse%2C%22cursor%22%3A%22YXJyYXljb25uZWN0aW9uOjk%3D%22%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%228334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8%22%7D%7D"
    
    response = session.get(url, headers=headers)
    print(f"Status: {response.status_code}")
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.text}")
        return None

# Test it
result = scrape_nytimes_with_session()
if result:
    print("Success!")
    print(json.dumps(result, indent=2)[:500])

Getting main page to establish session...
Status: 200
Success!
{
  "data": {
    "legacyCollection": {
      "__typename": "LegacyCollection",
      "active": true,
      "adTargetingParams": [
        {
          "__typename": "AdTargetingParam",
          "key": "tt",
          "value": ""
        },
        {
          "__typename": "AdTargetingParam",
          "key": "is_viral",
          "value": ""
        },
        {
          "__typename": "AdTargetingParam",
          "key": "prop",
          "value": "nyt"
        },
        {
          "__typen


In [20]:
#Example query
operation_name = "CollectionsQuery"
variables = {
    "id":"/section/business/media",
    "first":10,
    "exclusionMode":"HIGHLIGHTS_AND_EMBEDDED",
    "isFetchMore":False,
    "isTranslatable":False,
    "isEspanol":False,
    "highlightsListUri":"nyt://per/personalized-list/__null__",
    "highlightsListFirst":0,
    "hasHighlightsList":False,
    "cursor":"YXJyYXljb25uZWN0aW9uOjI5"
}
extension= {"persistedQuery":
                          {"version":1,"sha256Hash":"8334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8"}
                          }
proxies = {"http": "103.116.7.54"}

var_query = quote(json.dumps(variables))
extension_query = quote(json.dumps(extension))


url = f"https://samizdat-graphql.nytimes.com/graphql/v2?operationName={operation_name}&variables={var_query}&extensions={extension_query}"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.nytimes.com/section/business/media",
    "Origin": "https://www.nytimes.com",
    "Connection": "keep-alive",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors", 
    "Sec-Fetch-Site": "same-site",
    "sec-ch-ua": '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"'
}
r=requests.get(url=url, headers=headers, proxies=proxies)


r.raise_for_status()
r.json()

HTTPError: 403 Client Error: Forbidden for url: https://samizdat-graphql.nytimes.com/graphql/v2?operationName=CollectionsQuery&variables=%7B%22id%22%3A%20%22/section/business/media%22%2C%20%22first%22%3A%2010%2C%20%22exclusionMode%22%3A%20%22HIGHLIGHTS_AND_EMBEDDED%22%2C%20%22isFetchMore%22%3A%20false%2C%20%22isTranslatable%22%3A%20false%2C%20%22isEspanol%22%3A%20false%2C%20%22highlightsListUri%22%3A%20%22nyt%3A//per/personalized-list/__null__%22%2C%20%22highlightsListFirst%22%3A%200%2C%20%22hasHighlightsList%22%3A%20false%2C%20%22cursor%22%3A%20%22YXJyYXljb25uZWN0aW9uOjI5%22%7D&extensions=%7B%22persistedQuery%22%3A%20%7B%22version%22%3A%201%2C%20%22sha256Hash%22%3A%20%228334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8%22%7D%7D

In [93]:
import re
url = "https://samizdat-graphql.nytimes.com/graphql/v2?operationName=CollectionsQuery&variables=%7B%22id%22%3A%22%2Fsection%2Fbusiness%2Fmedia%22%2C%22first%22%3A10%2C%22exclusionMode%22%3A%22HIGHLIGHTS_AND_EMBEDDED%22%2C%22isFetchMore%22%3Afalse%2C%22isTranslatable%22%3Afalse%2C%22isEspanol%22%3Afalse%2C%22highlightsListUri%22%3A%22nyt%3A%2F%2Fper%2Fpersonalized-list%2F__null__%22%2C%22highlightsListFirst%22%3A0%2C%22hasHighlightsList%22%3Afalse%2C%22cursor%22%3A%22YXJyYXljb25uZWN0aW9uOjI5%22%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%228334262659d77fc2166184bf897e6d139e437af3a9b84d0c020d3dfcb0f177b8%22%7D%7D"
home_url = "https://nytimes.com"

proxies = {"http": "108.161.135.118"}
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",}

session = requests.Session()
session.proxies.update(proxies)
session.headers.update(headers)

r = session.get(url="https://nytimes.com",headers=headers)

token_search = re.search(r'["\']nyt-token["\']\s*:\s*["\']([^"\']+)["\']', r.text)

app_type_search = re.search(r'["\']nyt-app-type["\']\s*:\s*["\']([^"\']+)["\']', r.text)

app_type_version = re.search(r'["\']nyt-app-version["\']\s*:\s*["\']([^"\']+)["\']', r.text)

print(f"Status: {r.status_code}")
print(f"Response: {r.text[:200]}...")
print()
print()
print(f'Token search: {token_search.group()}')
print(f'App type: {app_type_search.group()}')
print(f'App version: {app_type_version.group()}')
#session = ex.create_client()




Status: 200
Response: <!DOCTYPE html>
<html lang="en" class=" nytapp-vi-homepage "  data-nyt-compute-assignment="fallback" xmlns:og="http://opengraphprotocol.org/schema/">
  <head>
    
    
    <meta charset="utf-8" />
  ...


Token search: "nyt-token":"MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAs+/oUCTBmD/cLdmcecrnBMHiU/pxQCn2DDyaPKUOXxi4p0uUSZQzsuq1pJ1m5z1i0YGPd1U1OeGHAChWtqoxC7bFMCXcwnE1oyui9G1uobgpm1GdhtwkR7ta7akVTcsF8zxiXx7DNXIPd2nIJFH83rmkZueKrC4JVaNzjvD+Z03piLn5bHWU6+w+rA+kyJtGgZNTXKyPh6EC6o5N+rknNMG5+CdTq35p8f99WjFawSvYgP9V64kgckbTbtdJ6YhVP58TnuYgr12urtwnIqWP9KSJ1e5vmgf3tunMqWNm6+AnsqNj8mCLdCuc5cEB74CwUeQcP2HQQmbCddBy2y0mEwIDAQAB"
App type: "nyt-app-type":"project-vi"
App version: "nyt-app-version":"0.0.5"
